In [ ]:
import os
import pandas as pd
from natsort import natsorted 
import numpy as np

#directory where data is stored
unzipped_data_dir = r"c:\Users\ramah\Desktop\DATA 608 Files\Flight_Route_Arrival_Inferences\Unzipped_Historical_Data"

# lists all csv files in directory
csv_files = [f for f in os.listdir(unzipped_data_dir) if f.endswith(".csv")]

#sorting the list of all csv fils in directory bcs they were out of order
csv_files = natsorted(csv_files)


In [ ]:
# selecting the first csv file 
sample_file = os.path.join(unzipped_data_dir, csv_files[0])
df = pd.read_csv(sample_file)

#dropping all cancelled flights
df = df[df["CANCELLED"] == 0]

In [ ]:
delay_cols = ["DEP_DELAY", "ARR_DELAY", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]

# replace nan with a 0
df[delay_cols] = df[delay_cols].fillna(0)

# summing delay column
df["Delay"] = df[delay_cols].sum(axis=1)

In [ ]:
#takes into consideration the timestamp (all the same i believe - 12:00AM)
df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")

# removes the timestamp
df["FL_DATE"] = df["FL_DATE"].dt.date

df["FL_DATE"] = pd.to_datetime(df["FL_DATE"])

In [ ]:
# assigning seasons depending on month
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9,10,11]:
        return "Fall"
    else:
        return "NA" #is this fine?

# dt.month extracts month from datetime column, and applies the above function into new "Season" column
df["Season"] = df["FL_DATE"].dt.month.apply(get_season)

In [ ]:
#categorizing time of day flight occured based on departure time
def categorize_time(dep_time):
    if 500 <= dep_time < 800:
        return "Early Morning"
    elif 800 <= dep_time < 1200:
        return " Late Morning"
    elif 1200 <= dep_time < 1500:
        return "Early Afternoon"
    elif 1500 <= dep_time < 1700:
        return "Late Afternoon"
    elif 1700 <= dep_time < 1900:
        return "Early Evening"
    elif 1900 <= dep_time < 2100:
        return "Late Evening"
    elif 2100 <= dep_time < 0:
        return "Night"


# applied categorize_time function to the dep_time column 
df["Time_of_Day"] = df["DEP_TIME"].apply(categorize_time)

In [ ]:
df["Route_Pair"] = df["ORIGIN"] + " to " + df["DEST"]

#grouping data by Route-Pair, Time of Day, Airline (Carrier), and Season and then calculating the mean and sd for their delay 
grouped_df = df.groupby(["Route_Pair", "Time_of_Day", "OP_CARRIER", "Season"])["Delay"].agg(["mean", "std", "count"]).reset_index()
grouped_df

In [ ]:
#second half of CI formula (1.96 for Z-score of 95% CI)
grouped_df["CI_2nd_Half"] = 1.96 * (grouped_df["std"] / np.sqrt(grouped_df["count"]))

#lower and upper bounds 
grouped_df["CI_Lower"] = grouped_df["mean"] - grouped_df["CI_2nd_Half"]
grouped_df["CI_Upper"] = grouped_df["mean"] + grouped_df["CI_2nd_Half"]


# Filter rows where count is 1 (aka only one flight in group so cant perform CI calculations)
count_ones_rows = grouped_df[grouped_df["count"] == 1]


In [ ]:
#example of flights from JFK to LAX

grouped_df[grouped_df["Route_Pair"] == "JFK to LAX"]